In [1]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import NetworkAnalysis

import pandas as pd
import numpy as np
import geopandas as gpd
from pydantic import BaseModel
import xarray as xr
import dfm_tools as dfmt
import xugrid as xu
import matplotlib.pyplot as plt
import random

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

import networkx as nx
import ribasim

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
base_dir = Path("c:\\Users\\NLHARN\\Documents\\SWECO_LOKAAL\\NHI-TKI-oppervlaktewatermodule-Ribasim\\TKI-NHI-oppervlaktewatermodule\\")
# base_dir = Path("C:\\Users\\NLTAND\\OneDrive - Sweco AB\\Algemeen-Tessa\\Projecten\\TKI oppervlaktewatermodule NHI\\")
simulations_dir = Path(base_dir, "d-hydro\\dhydro\\")
date_range = pd.date_range("2000-01-02 23:00", periods=9, freq="2D")

# load areas units (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "wrij_afwateringseenheden_clip_Zutphen\\wrij_afwateringseenheden_clip_Zutphen.shp")
areas = gpd.read_file(areas_file_path)[["geometry"]].to_crs(28992)

# output geopackage
gpkg_path = "output/test_export_geopackage/ribasim_network.gpkg"

In [6]:
# create networkanalysis
network = NetworkAnalysis(areas_gdf=areas)

# add data from simulation sets
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(simulations_dir, ""),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=date_range,
)
network.add_data_from_simulations_set(
    set_name="zomer",
    simulations_dir=Path(simulations_dir, ""),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=date_range,
)

# get all network data from the schematisations
network.get_network_data()

Simulation set (winter): tki_zuthpen_berkel_basis.dsproj | Timestamps: 9 | his.nc and map.nc
Simulation set (zomer): tki_zuthpen_berkel_basis.dsproj | Timestamps: 9 | his.nc and map.nc
network locations read: nodes/edges/confluences/bifurcations/weirs/pumps/laterals


In [46]:
# Define node_ids on which to split
# - define types to include
split_node_ids_1 = network.get_node_ids_from_type(
    bifurcations=False,
    confluences=False,
    weirs=True,
    laterals=False
)

# - define by node code
split_node_ids_2 = [1452, 2378]

# - define which of the node_ids should be excluded
split_node_ids_exclude = [314]

# - combine types of split_node_ids
split_node_ids = split_node_ids_1 + split_node_ids_2
split_node_ids = [node_id for node_id in split_node_ids if node_id not in split_node_ids_exclude]

In [47]:
# create basins (gdf) based on nodes, edges, split_node_ids and areas
results = network.create_basins_based_on_split_node_ids(split_node_ids=split_node_ids)
# export it to geopackage
network.export_to_geopackage(output=gpkg_path)

In [44]:
# get split_nodes in geodataframe
split_nodes_gdf = gpd.read_file(gpkg_path, layer='split_nodes')

In [45]:
# create basins (gdf) based on nodes, edges, split_node_ids and areas
results = network.create_basins_based_on_split_node_gdf(split_node_gdf=split_nodes_gdf)
# export it to geopackage
network.export_to_geopackage(output=gpkg_path)